![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png)
# Production code: logging, unit testing, refactoring, and PEP-8
Week 11 | Lesson 3.1


### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Implement a logging mechanism in Python
- Write unit tests and run them as a suite
- Identify examples of code that could be refactored

### Software engineer: You didn't check your code and your tests into master without a code review, did you?


### Data Scientist: Checked my what into what without a what?

#### - Software development skills for data scientists (http://treycausey.com/software_dev_skills.html)

### LESSON GUIDE
| TIMING  | TYPE  | TOPIC  |
|:-:|---|---|
| 5 min | [Opening](#opening) | Opening |
| 30 min | [Logging](#logging) | Logging |
| 30 min | [Unit testing](#unit-testing) | Unit testing |
| 20 min | [Refactoring](#refactoring) | Refactoring |
| 5 min | [Conclusion](#conclusion) | Conclusion |

### Logging

- Why log? Diagnostics and auditing.
- Ad hoc `print` statements: not crazy for simple personal scripts. Not sufficient for anything more

#### Python's `logging` library

- Better practice: more control, less mess

#### Codealong: `logging`

We'll add logging statements that output a .log file.

```python
import logging
logging.basicConfig(filename='test.log', level=logging.INFO)
log = logging.getLogger(__name__)
```

This configures our logging system (specifically, the "root logger"), by specifying a write file and a "level" (more on that in a moment). It also creates a logger object with a name of either `__main__` or the name of your module.

> The special variable `__name__` is equal to "`__main__`" when a program is run directly, or equal to the name of that _module_ when it's imported by another module. E.g. if `example.py` consists of the line `print __name__`, if you run `python example.py` you'll see `__main__`. If you run `python newModule.py` and `import example`, you'll see `example` printed.


```python
log.debug('Very granular logging message, useful for debugging.')
log.info('Simple update on normal execution, e.g. "Processing record {} of {}"'.format(10,100)
log.warning('You\'ve seen these in sklearn, warning about methods being deprecated')
log.error('Logs an error message')
log.critical('Well this is an issue')
```

There are five "levels" of logging. You  configure the system's threshold for logging via logging.basicConfig.

> Check: what do you expect to see in test.log?

There are other useful configuration parameters, particularly format:

```python
logging.basicConfig(filename='test.log', filemode='w', format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO)
```

You'll need to add this at the start of your file, as .basicConfig does nothing if it's run after your logging system is configured.

#### Configuring via JSON inputs

We've been hardcoding our logging settings. This is a relatively brittle approach. What if we want to make tweaks? What if we have multiple modules to configure? 

One alternative is to read in a JSON configuration file:

```json
{
    "version": 1,
    "disable_existing_loggers": "false",
    "formatters": {
        "basic": {
            "class": "logging.Formatter",
            "datefmt": "%Y-%m-%d %I:%M:%S",
            "format": "%(asctime)s %(levelname)s %(name)s %(message)s"
        }
    },

    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "level": "DEBUG",
            "formatter": "basic",
            "stream": "ext://sys.stdout"
        },
        "file": {
            "class": "logging.FileHandler",
            "level": "DEBUG",
            "formatter": "basic",
            "filename": "test.log",
            "mode": "w",
            "encoding": "utf-8"
        }
    },

    "loggers": { },

    "root": {
        "handlers": ["console", "file"],
        "level": "INFO"
    }
}
```

This example includes a few important keys:

- version: must be set to 1.
- disable_existing_loggers: defaults to True.
- formatters: sets the format for the logging messages
- handlers: configure the components that send log records to their destinations
- loggers: empty here, but you can name specific loggers
- root: configures the root logger, which all other loggers descend from

(There are other components explained in the documentation. https://docs.python.org/3/library/logging.config.html#logging-config-dictschema)

You can parse and pass the JSON file into `logging.config.dictConfig()`:

```python
import json

with open("logging_config.json", "r") as fd:
    logging.config.dictConfig(json.load(fd))
```

### Independent practice: logging

- Using the `logging` library, add at least three logging statements to one of the scripts you've written in class.
- Write a JSON file and use it to configure your logger
- Output the stream to 'test.log'. Slack your log!

## Unit testing

Testing for bugs is ubiquitous within software development. There are structured frameworks for doing this.

_Unit testing_ is testing the most granular components of your code, e.g. specific functions, to look for syntax, logic and execution errors.

If your job is data analysis, rather than building data products, you'll probably get away without formal testing. But it's still a good idea. It will sharpen your code, ease collaboration, and make _refactoring_ less fretful.   

There are several frameworks for unit testing in Python. We'll use `pytest` today:

```bash
pip install -U pytest
```

`pytest` is popular because it simplifies the code required to write and run tests. But you should also get familiar with the base [unittest/PyUnit library](https://docs.python.org/2/library/unittest.html).

Your initial tests can be written based on your program specifications: what are its functions supposed to _do_? Let's say we have these trivial functions:

```python

def rectangle_area(w,h):
    return w*h

def strip_stopwords(phrase, stopwords):
    phrase = phrase.split()
    phrase = [w for w in phrase if w not in stopwords]
    phrase = ' '.join(phrase)
    return phrase
```

If these were in a file (module) named `example.py`, then a pytest `test.py` file could look like:

```python
import pytest
import examples

def test_area_calculation():
    assert examples.rectangle_area(10,2) == 20
    
def test_stopwords():
    sentence = "the quick brown fox jumped over the lazy dog"
    stopwords = ['the', 'an', 'a', 'of', 'to']
    assert examples.strip_stopwords(sentence, stopwords) == 'quick brown fox jumped over lazy dog'
```

We could `assert` any Boolean condition, e.g.:
```python
assert examples.strip_stopwords(sentence, stopwords) != 'the quick brown fox jumped over the lazy dog'
```

It is also important to consider what your code should _not_ do, i.e., when should it fail and what exceptions should it raise? Our area calculation function should only work with numeric types:

```python
def test_area_type_handling():
    with pytest.raises(TypeError):
        examples.rectangle_area(5,'testing')
```

`assert` and `with pytest.raises(___Error): ....` are two workhorse commands.

### Guided practice: running a test suite

A basic unit testing battery requires a couple things:

- A script with your test functions, each of which has a name starting with "test"
- The module you want to test (for simplicity, in the same directory)

Py.test will automatically detect and run your tests for you. Let's try it! The bash command is:

```bash
py.test [-v] test_script.py
```

Often you will need to test methods of classes - and for this you may need to instantiate the class with specific values. Take yesterday's Tic Tac Toe code, for example:

```python
import random
import numpy as np

class tictactoe():
    def __init__(self):
        self.board = np.chararray((3,3))
        self.wincombos = [self.board[0], self.board[1], self.board[2], \
                          self.board[:,0], self.board[:,1], self.board[:,2], \
                          self.board.diagonal(), np.fliplr(self.board).diagonal()]
        
        keyboard = np.array([[1,2,3],[4,5,6],[7,8,9]])
        moves_dict = {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1), 6:(1,2), 7:(2,0), 8:(2,1), 9:(2,2)}
        
    def make_computer_move(self):
        '''Only makes random moves. 

        If you want to improve the game's AI, start here.
        '''
        
        #Make a random move 
        x = random.randint(0,2)
        y = random.randint(0,2)

        #Check if that space is occupied
        while self.board[x][y] != '-':
            x = random.randint(0,2)
            y = random.randint(0,2)
            
        self.board[x][y] = 'O'
 ```

Let's say we want to make sure `make_computer_move` puts an 'O' into a board full of '-'s. Note that the initialization does _not_ include setting up the board with dashes. So how do we test this function?

"Fixtures". They're a little involved, but the basic syntax is:

```python
@pytest.fixture()
def setup():
	t = tictactoe.tictactoe()
	t.board[:] = '-'
	return t

def test_computer_move(setup):
	setup.make_computer_move()
	assert 'O' in setup.board
```


### Independent practice: writing test functions

Let's take a step toward production! Add at least one more unit test for the trivial example functions, and two more unit tests for the Tic Tac Toe solution code (or your version). 

### Refactoring

This software development vocabulary word just means "improving your code". The general axes are:

- Efficiency
- Readability
- Extensibility

Some easy wins:

- Don't Repeat Yourself (DRY)
- Use helpful names
- Comment your code!

You can also improve your code's readability, and your own credibility, by following a community standard stylistic convention. The most popular is [PEP-8](https://www.python.org/dev/peps/pep-0008/).

Please take a few minutes to skim the documentation.

"Linters" are tools for checking your code for errors. There are style linters available, as standalone programs or integrations to IDEs / text editors.

We'll use an easy one:

```bash
$pip install -U pep8
$pep8 tictactoe.py
```

Let's look at a few refactoring examples together:

```python
if isSpecialDeal():
    total = price * 0.95
    send()
else:
    total = price * 0.98
    send()
```

> Check: what notion is this violating? How can we improve it?


(Examples from https://github.com/shvetsgroup/refactoring.guru-examples/tree/master/simple/python)

Don't repeat yourself:

```python
if isSpecialDeal():
    total = price * 0.95
else:
    total = price * 0.98
send()
```

How about this one?

```python
def output(self, type):
    if name == "banner"
        # Print the banner.
        # ...
    if name == "info"
        # Print the info.
        # ...
```



Make it easier to adjust what happens in each case:

```python
def outputBanner(self):
    # Print the banner.
    # ...

def outputInfo(self):
    # Print the info.
    # ...
```

And here?

```python
def foundPerson(people):
    for i in range(len(people)):
        if people[i] == "Don":
            return "Don"
        if people[i] == "John":
            return "John"
        if people[i] == "Kent":
            return "Kent"
    return ""
```

That code wasn't very Pythonic, plus it doubles the risk of typos.

```python
def foundPerson(people):
    candidates = ["Don", "John", "Kent"]
    for i in range(len(people)):
        if people[i] in candidates:
            return people[i]
    return ""
```

### Additional resources

Software development

- http://treycausey.com/software_dev_skills.html
- http://12factor.net/

Logging

- http://victorlin.me/posts/2012/08/26/good-logging-practice-in-python
- http://www.blog.pythonlibrary.org/2012/08/02/python-101-an-intro-to-logging/

(Unit) testing
- http://docs.python-guide.org/en/latest/writing/tests/
- http://stackoverflow.com/questions/4904096/whats-the-difference-between-unit-functional-acceptance-and-integration-test
